In [ ]:
!pip install pmdarima

import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_datareader import data as pdr
import yfinance as yf

from sklearn.preprocessing import MinMaxScaler
from pandas.plotting import lag_plot
from pandas import datetime
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt

# Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
bmw5 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CS346/car/BMW5.csv')
rr5 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CS346/car/RYCEY.csv')
hyun5 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CS346/car/HYMTF.csv')
bosch5 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CS346/car/BOSCH5.csv')

# Check missing value

In [ ]:
bm = bmw5.dropna()

In [ ]:
rr = rr5.dropna()

In [ ]:
hy = hyun5.dropna()

In [ ]:
bos = bosch5.dropna()

In [ ]:
bm.info()

In [ ]:
rr.info()

In [ ]:
hy.info()

In [ ]:
bos.info()

In [ ]:
company_list = [bm, rr, hy, bos]
company_name = ["BMW", "Rolls-Royce", "Hyundai", "BOSCH"]

tech_list = ['BMW', 'Rolls-Royce', 'Hyundai', 'BOSCH']

In [ ]:
plt.figure(figsize=(15, 10))
plt.subplots_adjust(top=1.35, bottom=1.2)

for i, company in enumerate(company_list, 1):
  plt.subplot(2, 2, i)
  company['Close'].plot()
  plt.ylabel('Close')
  plt.xlabel(None)
  plt.title(f"Closing Price of {tech_list[i - 1]}")

plt.tight_layout()

In [ ]:
plt.figure(figsize=(15, 10))
plt.subplots_adjust(top=1.35, bottom=1.2)

for i, company in enumerate(company_list, 1):
  plt.subplot(2, 2, i)
  company['Volume'].plot()
  plt.ylabel('Volume')
  plt.xlabel(None)
  plt.title(f"Sales Volume for {tech_list[i - 1]}")

plt.tight_layout()

In [ ]:
plt.figure(figsize=(12, 9))

for i, company in enumerate(company_list, 1):
    plt.subplot(2, 2, i)
    company['Adj Close'].hist(bins=50)
    plt.xlabel('Adj Close')
    plt.ylabel('Counts')
    plt.title(f'{company_name[i - 1]}')
    
plt.tight_layout()

In [ ]:
plt.figure(figsize=(12, 9))

for i, company in enumerate(company_list, 1):
    plt.subplot(2, 2, i)
    company['Adj Close'].hist(bins=50)
    plt.xlabel('Adj Close')
    plt.ylabel('Counts')
    plt.title(f'{company_name[i - 1]}')
    
plt.tight_layout()

In [ ]:
plt.figure()
lag_plot(bm['Close'], lag=3)
plt.title('BMW Stock - Autocorrelation plot with lag = 3')
plt.show()

In [ ]:
plt.figure()
lag_plot(rr['Close'], lag=3)
plt.title('ROLL_ROYCE Stock - Autocorrelation plot with lag = 3')
plt.show()

In [ ]:
plt.figure()
lag_plot(hy['Close'], lag=3)
plt.title('HYUNDAI Stock - Autocorrelation plot with lag = 3')
plt.show()

In [ ]:
plt.figure()
lag_plot(bos['Close'], lag=3)
plt.title('BOSCH Stock - Autocorrelation plot with lag = 3')
plt.show()

In [ ]:
plt.plot(bm["Date"], bm["Close"])
plt.xticks(np.arange(0,1259, 200), bm['Date'][0:1259:200])
plt.title("BMW stock price over time")
plt.xlabel("time")
plt.ylabel("price")
plt.show()

In [ ]:
plt.figure(figsize = (18,9))
plt.plot(range(bm.shape[0]),(bm['Close']))
plt.xticks(range(0,bm.shape[0],500),bm['Date'].loc[::500],rotation=45)
plt.xlabel('Date',fontsize=18)
plt.ylabel('BMW stock price over time',fontsize=18)
plt.show()

In [ ]:
plt.figure(figsize = (18,9))
plt.plot(range(rr.shape[0]),(rr['Close']))
plt.xticks(range(0,rr.shape[0],500),rr['Date'].loc[::500],rotation=45)
plt.xlabel('Date',fontsize=18)
plt.ylabel('ROLL ROYCE stock price over time',fontsize=18)
plt.show()

In [ ]:
plt.figure(figsize = (18,9))
plt.plot(range(hy.shape[0]),(hy['Close']))
plt.xticks(range(0,hy.shape[0],500),hy['Date'].loc[::500],rotation=45)
plt.xlabel('Date',fontsize=18)
plt.ylabel('HYUNDAI stock price over time',fontsize=18)
plt.show()

In [ ]:
plt.figure(figsize = (18,9))
plt.plot(range(bos.shape[0]),(bos['Close']))
plt.xticks(range(0,bos.shape[0],500),bos['Date'].loc[::500],rotation=45)
plt.xlabel('Date',fontsize=18)
plt.ylabel('BOSCH stock price over time',fontsize=18)
plt.show()

In [ ]:
dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m-%d')
#stock_data = pd.read_csv('acgl.us.txt',sep=',', index_col='Date', parse_dates=['Date'], date_parser=dateparse).fillna(0)


bm = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CS346/car/BMW5.csv',sep=',', index_col='Date', parse_dates=['Date'], date_parser=dateparse).dropna()
rr = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CS346/car/RYCEY.csv',sep=',', index_col='Date', parse_dates=['Date'], date_parser=dateparse).dropna()
hy = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CS346/car/HYMTF.csv',sep=',', index_col='Date', parse_dates=['Date'], date_parser=dateparse).dropna()
bos = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CS346/car/BOSCH5.csv',sep=',', index_col='Date', parse_dates=['Date'], date_parser=dateparse).dropna()

bm

In [ ]:
rr

In [ ]:
hy

In [ ]:
bos

In [ ]:
#plot close price
plt.figure(figsize=(10,6))
plt.grid(True)
plt.xlabel('Date')
plt.ylabel('Close Prices')
plt.plot(bm['Close'])
plt.title('BMW closing price')
plt.show()

In [ ]:
#plot close price
plt.figure(figsize=(10,6))
plt.grid(True)
plt.xlabel('Date')
plt.ylabel('Close Prices')
plt.plot(rr['Close'])
plt.title('ROLL_ROYCE closing price')
plt.show()

In [ ]:
#plot close price
plt.figure(figsize=(10,6))
plt.grid(True)
plt.xlabel('Date')
plt.ylabel('Close Prices')
plt.plot(hy['Close'])
plt.title('HYUNDAI closing price')
plt.show()

In [ ]:
#plot close price
plt.figure(figsize=(10,6))
plt.grid(True)
plt.xlabel('Date')
plt.ylabel('Close Prices')
plt.plot(bos['Close'])
plt.title('BOSCH closing price')
plt.show()

In [ ]:
#Distribution of the dataset
bm_close = bm['Close']
bm_close.plot(kind='kde', title='BMW')

In [ ]:
#Distribution of the dataset
rr_close = rr['Close']
rr_close.plot(kind='kde', title='ROLL ROYCE')

In [ ]:
#Distribution of the dataset
hy_close = hy['Close']
hy_close.plot(kind='kde', title='HYUNDAI')

In [ ]:
#Distribution of the dataset
bos_close = bos['Close']
bos_close.plot(kind='kde', title='BOSCH')

# ADF Test

In [ ]:
#Test for staionarity
def test_stationarity(timeseries):
    #Determing rolling statistics
    rolmean = timeseries.rolling(12).mean()
    rolstd = timeseries.rolling(12).std()
    #Plot rolling statistics:
    plt.plot(timeseries, color='blue',label='Original')
    plt.plot(rolmean, color='red', label='Rolling Mean')
    plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean and Standard Deviation - BMW')
    plt.show(block=False)
    print("Results of dickey fuller test")
    adft = adfuller(timeseries,autolag='AIC')
    # output for dft will give us without defining what the values are.
    #hence we manually write what values does it explains using a for loop
    output = pd.Series(adft[0:4],index=['Test Statistics','p-value','No. of lags used','Number of observations used'])
    for key,values in adft[4].items():
        output['critical value (%s)'%key] =  values
    print(output)
test_stationarity(bm_close)

In [ ]:
#Test for staionarity
def test_stationarity(timeseries):
    #Determing rolling statistics
    rolmean = timeseries.rolling(12).mean()
    rolstd = timeseries.rolling(12).std()
    #Plot rolling statistics:
    plt.plot(timeseries, color='blue',label='Original')
    plt.plot(rolmean, color='red', label='Rolling Mean')
    plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean and Standard Deviation - ROLL ROYCE')
    plt.show(block=False)
    print("Results of dickey fuller test")
    adft = adfuller(timeseries,autolag='AIC')
    # output for dft will give us without defining what the values are.
    #hence we manually write what values does it explains using a for loop
    output = pd.Series(adft[0:4],index=['Test Statistics','p-value','No. of lags used','Number of observations used'])
    for key,values in adft[4].items():
        output['critical value (%s)'%key] =  values
    print(output)
test_stationarity(rr_close)

In [ ]:
#Test for staionarity
def test_stationarity(timeseries):
    #Determing rolling statistics
    rolmean = timeseries.rolling(12).mean()
    rolstd = timeseries.rolling(12).std()
    #Plot rolling statistics:
    plt.plot(timeseries, color='blue',label='Original')
    plt.plot(rolmean, color='red', label='Rolling Mean')
    plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean and Standard Deviation - HYUNDAI')
    plt.show(block=False)
    print("Results of dickey fuller test")
    adft = adfuller(timeseries,autolag='AIC')
    # output for dft will give us without defining what the values are.
    #hence we manually write what values does it explains using a for loop
    output = pd.Series(adft[0:4],index=['Test Statistics','p-value','No. of lags used','Number of observations used'])
    for key,values in adft[4].items():
        output['critical value (%s)'%key] =  values
    print(output)
test_stationarity(hy_close)

In [ ]:
#Test for staionarity
def test_stationarity(timeseries):
    #Determing rolling statistics
    rolmean = timeseries.rolling(12).mean()
    rolstd = timeseries.rolling(12).std()
    #Plot rolling statistics:
    plt.plot(timeseries, color='blue',label='Original')
    plt.plot(rolmean, color='red', label='Rolling Mean')
    plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean and Standard Deviation - BOSCH')
    plt.show(block=False)
    print("Results of dickey fuller test")
    adft = adfuller(timeseries,autolag='AIC')
    # output for dft will give us without defining what the values are.
    #hence we manually write what values does it explains using a for loop
    output = pd.Series(adft[0:4],index=['Test Statistics','p-value','No. of lags used','Number of observations used'])
    for key,values in adft[4].items():
        output['critical value (%s)'%key] =  values
    print(output)
test_stationarity(bos_close)

In [ ]:
#To separate the trend and the seasonality from a time series, 
# we can decompose the series using the following code.
bm_sea = seasonal_decompose(bm_close, model='multiplicative', period = 30)
fig = plt.figure()  
fig = bm_sea.plot()  
fig.set_size_inches(16, 9)

In [ ]:
#To separate the trend and the seasonality from a time series, 
# we can decompose the series using the following code.
rr_sea = seasonal_decompose(rr_close, model='multiplicative', period = 30)
fig = plt.figure()  
fig = rr_sea.plot()  
fig.set_size_inches(16, 9)

In [ ]:
#To separate the trend and the seasonality from a time series, 
# we can decompose the series using the following code.
hy_sea = seasonal_decompose(hy_close, model='multiplicative', period = 30)
fig = plt.figure()  
fig = hy_sea.plot()  
fig.set_size_inches(16, 9)

In [ ]:
#To separate the trend and the seasonality from a time series, 
# we can decompose the series using the following code.
bos_sea = seasonal_decompose(bos_close, model='multiplicative', period = 30)
fig = plt.figure()  
fig = bos_sea.plot()  
fig.set_size_inches(16, 9)

In [ ]:
#if not stationary then eliminate trend
#Eliminate trend
from pylab import rcParams
rcParams['figure.figsize'] = 10, 6
bm_log = np.log(bm_close)
moving_avg = bm_log.rolling(12).mean()
std_dev = bm_log.rolling(12).std()
plt.legend(loc='best')
plt.title('Moving Average BMW')
plt.plot(std_dev, color ="black", label = "Standard Deviation")
plt.plot(moving_avg, color="red", label = "Mean")
plt.legend()
plt.show()

In [ ]:
#if not stationary then eliminate trend
#Eliminate trend
from pylab import rcParams
rcParams['figure.figsize'] = 10, 6
rr_log = np.log(bm_close)
moving_avg = rr_log.rolling(12).mean()
std_dev = rr_log.rolling(12).std()
plt.legend(loc='best')
plt.title('Moving Average ROLL ROYCE')
plt.plot(std_dev, color ="black", label = "Standard Deviation")
plt.plot(moving_avg, color="red", label = "Mean")
plt.legend()
plt.show()

In [ ]:
#if not stationary then eliminate trend
#Eliminate trend
from pylab import rcParams
rcParams['figure.figsize'] = 10, 6
hy_log = np.log(bm_close)
moving_avg = hy_log.rolling(12).mean()
std_dev = hy_log.rolling(12).std()
plt.legend(loc='best')
plt.title('Moving Average HYUNDAI')
plt.plot(std_dev, color ="black", label = "Standard Deviation")
plt.plot(moving_avg, color="red", label = "Mean")
plt.legend()
plt.show()

In [ ]:
#if not stationary then eliminate trend
#Eliminate trend
from pylab import rcParams
rcParams['figure.figsize'] = 10, 6
bos_log = np.log(bos_close)
moving_avg = bos_log.rolling(12).mean()
std_dev = bos_log.rolling(12).std()
plt.legend(loc='best')
plt.title('Moving Average BOSCH')
plt.plot(std_dev, color ="black", label = "Standard Deviation")
plt.plot(moving_avg, color="red", label = "Mean")
plt.legend()
plt.show()

In [ ]:
#split data into train and training set
train_bm, test_bm = bm_log[3:int(len(bm_log)*0.9)], bm_log[int(len(bm_log)*0.9):]
plt.figure(figsize=(10,6))
plt.grid(True)
plt.xlabel('Dates')
plt.ylabel('Closing Prices')
plt.title('BMW')
plt.plot(bm_log, 'green', label='Train data')
plt.plot(test_bm, 'blue', label='Test data')
plt.legend()

In [ ]:
#split data into train and training set
train_rr, test_rr = bm_log[3:int(len(rr_log)*0.9)], rr_log[int(len(rr_log)*0.9):]
plt.figure(figsize=(10,6))
plt.grid(True)
plt.xlabel('Dates')
plt.ylabel('Closing Prices')
plt.title('ROLL ROYCE')
plt.plot(rr_log, 'green', label='Train data')
plt.plot(test_rr, 'blue', label='Test data')
plt.legend()

In [ ]:
#split data into train and training set
train_hy, test_hy = hy_log[3:int(len(hy_log)*0.9)], hy_log[int(len(hy_log)*0.9):]
plt.figure(figsize=(10,6))
plt.grid(True)
plt.xlabel('Dates')
plt.ylabel('Closing Prices')
plt.title('HYUNDAI')
plt.plot(hy_log, 'green', label='Train data')
plt.plot(test_hy, 'blue', label='Test data')
plt.legend()

In [ ]:
#split data into train and training set
train_bos, test_bos = bos_log[3:int(len(bos_log)*0.9)], bos_log[int(len(bos_log)*0.9):]
plt.figure(figsize=(10,6))
plt.grid(True)
plt.xlabel('Dates')
plt.ylabel('Closing Prices')
plt.title('BOSCH')
plt.plot(bos_log, 'green', label='Train data')
plt.plot(test_bos, 'blue', label='Test data')
plt.legend()

In [ ]:
model_autoARIMA = auto_arima(train_bm, start_p=0, start_q=0,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)
print(model_autoARIMA.summary())
model_autoARIMA.plot_diagnostics(figsize=(15,8))
plt.show()

In [ ]:
model_autoARIMA = auto_arima(train_rr, start_p=0, start_q=0,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)
print(model_autoARIMA.summary())
model_autoARIMA.plot_diagnostics(figsize=(15,8))
plt.show()

In [ ]:
model_autoARIMA = auto_arima(train_hy, start_p=0, start_q=0,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)
print(model_autoARIMA.summary())
model_autoARIMA.plot_diagnostics(figsize=(15,8))
plt.show()

In [ ]:
model_autoARIMA = auto_arima(train_bos, start_p=0, start_q=0,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)
print(model_autoARIMA.summary())
model_autoARIMA.plot_diagnostics(figsize=(15,8))
plt.show()

# Generate train and test 

In [ ]:
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_pacf

In [ ]:
#Modeling
# Build Model
model_bm = sm.tsa.arima.ARIMA(train_bm, order=(1,1,2))  
fitted = model_bm.fit()  
print(fitted.summary())

In [ ]:
#Modeling
# Build Model
model_rr = sm.tsa.arima.ARIMA(train_rr, order=(1,1,2))  
fitted = model_rr.fit()  
print(fitted.summary())

In [ ]:
#Modeling
# Build Model
model_hy = sm.tsa.arima.ARIMA(train_hy, order=(1,1,2))  
fitted = model_hy.fit()  
print(fitted.summary())

In [ ]:
#Modeling
# Build Model
model_bos = sm.tsa.arima.ARIMA(train_bos, order=(1,1,2))  
fitted = model_bos.fit()  
print(fitted.summary())

# Forecasting and Evaluation metrics

In [ ]:
train_bm, test_bm = bm[0:int(len(bm)*0.7)], bm[int(len(bm)*0.7):]
training_bm = train_bm['Close'].values
test_bm = test_bm['Close'].values
history = [x for x in training_bm]
model_predictions = []
N_test_observations = len(test_bm)
for time_point in range(N_test_observations):
    model = sm.tsa.arima.ARIMA(history, order=(4,1,0))
    model_fit = model.fit()
    output = model_fit.forecast()
    yhat = output[0]
    model_predictions.append(yhat)
    true_test_value = test_bm[time_point]
    history.append(true_test_value)

#MSE_train = mean_squared_error(training_bm, model_predictions)
#RMSE_train = sqrt(MSE_train)
#MAE_train =mean_absolute_error(training_bm, model_predictions)

MSE_test = mean_squared_error(test_bm, model_predictions)
RMSE_test = sqrt(MSE_test)
MAE_test =mean_absolute_error(test_bm, model_predictions)

#print('Training Root Mean Squared Error is {}'.format(RMSE_train))
#print('Training Mean Squared Error is {}'.format(MSE_train))
#print('Training Mean Absolute Error is {}'.format(MAE_train))

MSE = f"{MSE_test:.4f}"
RMSE = f"{RMSE_test:.4f}"
MAE = f"{MAE_test:.4f}"

print('Testing Root Mean Squared Error is {}'.format(RMSE))
print('Testing Mean Squared Error is {}'.format(MSE))
print('Testing Mean Absolute Error is {}'.format(MAE))


In [ ]:
test_set_range = bm[int(len(bm)*0.7):].index
plt.plot(test_set_range, model_predictions, color='blue', marker='o', linestyle='dashed',label='Predicted Price')
plt.plot(test_set_range, test_bm, color='red', label='Actual Price')
plt.title('BMW Prices Prediction')
plt.xlabel('Date')
plt.ylabel('Prices')
plt.xticks(np.arange(881,1259,50), bm.Date[881:1259:50])
plt.legend()
plt.show()

In [ ]:
train_rr, test_rr = rr[0:int(len(rr)*0.7)], rr[int(len(rr)*0.7):]
training_rr = train_rr['Close'].values
test_rr = test_rr['Close'].values
history = [x for x in training_rr]
model_predictions = []
N_test_observations = len(test_rr)
for time_point in range(N_test_observations):
    model = sm.tsa.arima.ARIMA(history, order=(4,1,0))
    model_fit = model.fit()
    output = model_fit.forecast()
    yhat = output[0]
    model_predictions.append(yhat)
    true_test_value = test_rr[time_point]
    history.append(true_test_value)

MSE_error = mean_squared_error(test_rr, model_predictions)
RMSE_error = sqrt(MSE_error)
MAE_error =mean_absolute_error(test_rr, model_predictions)

MSE = f"{MSE_error:.4f}"
RMSE = f"{RMSE_error:.4f}"
MAE = f"{MAE_error:.4f}"

print('Testing Root Mean Squared Error is {}'.format(RMSE))
print('Testing Mean Squared Error is {}'.format(MSE))
print('Testing Mean Absolute Error is {}'.format(MAE))


In [ ]:
test_set_range = rr[int(len(rr)*0.7):].index
plt.plot(test_set_range, model_predictions, color='blue', marker='o', linestyle='dashed',label='Predicted Price')
plt.plot(test_set_range, test_rr, color='red', label='Actual Price')
plt.title('ROLL_ROYCE Prices Prediction')
plt.xlabel('Date')
plt.ylabel('Prices')
plt.xticks(np.arange(881,1259,50), rr.Date[881:1259:50])
plt.legend()
plt.show()

In [ ]:
train_hy, test_hy = hy[0:int(len(hy)*0.7)], hy[int(len(hy)*0.7):]
training_hy = train_hy['Close'].values
test_hy = test_hy['Close'].values
history = [x for x in training_hy]
model_predictions = []
N_test_observations = len(test_hy)
for time_point in range(N_test_observations):
    model = sm.tsa.arima.ARIMA(history, order=(4,1,0))
    model_fit = model.fit()
    output = model_fit.forecast()
    yhat = output[0]
    model_predictions.append(yhat)
    true_test_value = test_bm[time_point]
    history.append(true_test_value)

MSE_error = mean_squared_error(test_hy, model_predictions)
RMSE_error = sqrt(MSE_error)
MAE_error =mean_absolute_error(test_hy, model_predictions)

MSE = f"{MSE_error:.4f}"
RMSE = f"{RMSE_error:.4f}"
MAE = f"{MAE_error:.4f}"

print('Testing Root Mean Squared Error is {}'.format(RMSE))
print('Testing Mean Squared Error is {}'.format(MSE))
print('Testing Mean Absolute Error is {}'.format(MAE))


In [ ]:
test_set_range = hy[int(len(hy)*0.7):].index
plt.plot(test_set_range, model_predictions, color='blue', marker='o', linestyle='dashed',label='Predicted Price')
plt.plot(test_set_range, test_hy, color='red', label='Actual Price')
plt.title('HYUNDAI Prices Prediction')
plt.xlabel('Date')
plt.ylabel('Prices')
plt.xticks(np.arange(881,1259,50), hy.Date[881:1259:50])
plt.legend()
plt.show()

In [ ]:
train_bos, test_bos = bos[0:int(len(bos)*0.7)], bos[int(len(bos)*0.7):]
training_bos = train_bos['Close'].values
test_bos = test_bos['Close'].values
history = [x for x in training_bos]
model_predictions = []
N_test_observations = len(test_bos)
for time_point in range(N_test_observations):
    model = sm.tsa.arima.ARIMA(history, order=(4,1,0))
    model_fit = model.fit()
    output = model_fit.forecast()
    yhat = output[0]
    model_predictions.append(yhat)
    true_test_value = test_bos[time_point]
    history.append(true_test_value)

MSE_error = mean_squared_error(test_bos, model_predictions)
RMSE_error = sqrt(MSE_error)
MAE_error =mean_absolute_error(test_bos, model_predictions)

MSE = f"{MSE_error:.4f}"
RMSE = f"{RMSE_error:.4f}"
MAE = f"{MAE_error:.4f}"

print('Testing Root Mean Squared Error is {}'.format(RMSE))
print('Testing Mean Squared Error is {}'.format(MSE))
print('Testing Mean Absolute Error is {}'.format(MAE))

In [ ]:
test_set_range = bos[int(len(bos)*0.7):].index
plt.plot(test_set_range, model_predictions, color='blue', marker='o', linestyle='dashed',label='Predicted Price')
plt.plot(test_set_range, test_bos, color='red', label='Actual Price')
plt.title('BOSCH Prices Prediction')
plt.xlabel('Date')
plt.ylabel('Prices')
plt.xticks(np.arange(881,1259,50), bos.Date[881:1259:50])
plt.legend()
plt.show()